In [1]:
import imageio
import sys
import numpy as np
import visvis as vv
from skimage import color, measure, morphology
from skimage.feature import canny
from scipy import ndimage
from skvideo.io import VideoCapture
import matplotlib.pyplot as plt
import pylab
from scipy import ndimage as ndi
from skimage.filters import sobel
from skimage.morphology import watershed
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label
from skimage.morphology import closing, square
from skimage.measure import regionprops
from skimage.color import label2rgb
import matplotlib.patches as mpatches


In [2]:
def readvideo(src):
    return imageio.get_reader(src)

In [3]:
vid = imageio.get_reader('../sailfin_3_300_F_Hillary_1_4_5_L.mpg', 'ffmpeg')

In [4]:
framix = 200#np.linspace(120,len(vid),1)
#for num in framix:
frame = vid.get_data(framix)
earlyframe = vid.get_data(3000)
#lateframe = vid.get_data(100)
midframe = vid.get_data(1000)
earlyframe = color.rgb2gray(earlyframe)
midframe = color.rgb2gray(midframe)
greyframe = color.rgb2gray(frame)
avg = earlyframe+midframe+greyframe/3

In [77]:
print frame.shape

(1080, 1920, 3)


In [35]:
frame = color.rgb2gray(frame)
edges = canny(frame/255.)
fill = ndi.binary_fill_holes(edges)
label_objects, nb_levels = ndi.label(fill)
sizes = np.bincount(label_objects.ravel())
mask_sizes = sizes > 100
mask_sizes[0] = 0
clean_fill = mask_sizes[label_objects]
plt.imshow(edges)
plt.show()

In [62]:
histo = np.histogram(frame, bins=np.arange(0, 100))
sobf = sobel(frame)
markers = np.zeros_like(frame)
markers[frame < 2] = 2
markers[frame > 2 ] = 1
segm = watershed(sobf, markers)
plt.hist(histo)
#plt.imshow(segm)
plt.show()

In [8]:
frame = color.rgb2gray(frame)
frame = abs(frame-avg)
thresh = threshold_otsu(frame)
bw = closing(frame > thresh, square(3))
cleared = bw.copy()
clear_border(cleared)
label_image = label(cleared)
borders = np.logical_xor(bw, cleared)
label_image[borders] = -1
image_label_overlay = label2rgb(label_image, image=frame)

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6, 6))
ax.imshow(image_label_overlay)
for region in regionprops(label_image):

    # skip small images
    if region.area < 100:
        continue

    # draw rectangle around segmented coins
    minr, minc, maxr, maxc = region.bbox
    rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                              fill=False, edgecolor='red', linewidth=2)
    ax.add_patch(rect)
plt.show()